<a href="https://colab.research.google.com/github/EleMental-Capstone/EleMental-Machine-Learning/blob/models_trial/LSTM_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.layers import LSTM, Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

# Load the dataset
data = pd.read_csv("https://raw.githubusercontent.com/adi31891/capstone-C241-PR565/f8a43716e964aa3bc19b1de1d1947aa550dd6bb9/data/capstone_new_dataset.csv")

# Extract text and labels
texts = data['text'].astype(str).values
labels = data['sentiment'].astype(str).values

# Encode labels to integers
label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(labels)

# One-hot encode the integer labels
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded_labels = integer_encoded_labels.reshape(len(integer_encoded_labels), 1)
onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded_labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, onehot_encoded_labels, test_size=0.2, random_state=42)

# Load the Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Function to create embeddings for a given list of texts in batches
def get_embeddings_in_batches(texts, batch_size=256):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_embeddings = embed(batch_texts)
        embeddings.append(batch_embeddings)
    return tf.concat(embeddings, axis=0)

# Create embeddings for train and test sets in batches
train_embeddings = get_embeddings_in_batches(X_train)
test_embeddings = get_embeddings_in_batches(X_test)



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Define the LSTM model
input_shape = train_embeddings.shape[1]
inputs = Input(shape=(input_shape,), dtype=tf.float32)
x = tf.expand_dims(inputs, -1)
x = LSTM(32, return_sequences=True)(x)
x = BatchNormalization()(x)
x = LSTM(16)(x)
# x = Dropout(0.2)(x)
x = BatchNormalization()(x)
outputs = Dense(3, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_embeddings, y_train, epochs=20, batch_size=32, validation_data=(test_embeddings, y_test))



Epoch 1/20
5743/5743 [==============================] - 204s 34ms/step - loss: 1.0188 - accuracy: 0.5025 - val_loss: 1.1476 - val_accuracy: 0.3520
Epoch 2/20
5743/5743 [==============================] - 195s 34ms/step - loss: 1.0027 - accuracy: 0.5165 - val_loss: 1.1402 - val_accuracy: 0.4924
Epoch 3/20
5743/5743 [==============================] - 196s 34ms/step - loss: 0.9886 - accuracy: 0.5276 - val_loss: 1.1132 - val_accuracy: 0.4948
Epoch 4/20
5743/5743 [==============================] - 195s 34ms/step - loss: 0.9783 - accuracy: 0.5343 - val_loss: 1.0803 - val_accuracy: 0.4461
Epoch 5/20
5743/5743 [==============================] - 196s 34ms/step - loss: 0.9751 - accuracy: 0.5360 - val_loss: 1.0927 - val_accuracy: 0.4906
Epoch 6/20
5743/5743 [==============================] - 198s 35ms/step - loss: 0.9678 - accuracy: 0.5412 - val_loss: 1.0135 - val_accuracy: 0.5137
Epoch 7/20
5743/5743 [==============================] - 198s 35ms/step - loss: 0.9655 - accuracy: 0.5425 - val_loss: 1

In [ ]:
# Save the model architecture as JSON
model_json = model.to_json()
with open("lstm_model.json", "w") as json_file:
    json_file.write(model_json)

# Save the model weights
model.save_weights("lstm_model_weights.h5")

# Save Universal Sentence Encoder weights to .bin file
# Get all variables from the module
variables = embed.variables
with open('use_model_weights.bin', 'wb') as f:
    for var in variables:
        np.array(var).tofile(f)

# Function to predict sentiment
def predict_sentiment(text):
    embedding = embed([text])
    prediction = model.predict(embedding)
    sentiment_index = tf.argmax(prediction[0]).numpy()
    sentiment = label_encoder.inverse_transform([sentiment_index])[0]
    return sentiment

# Example usage with Gen AI response
def generate_response(sentiment):
    if sentiment == 'good':
        return "I'm glad to hear that! Keep up the positive vibes."
    elif sentiment == 'bad':
        return "I'm sorry to hear that. Remember, it's okay to feel this way sometimes. Can I help with something?"
    elif sentiment == 'neutral':
        return "Thanks for sharing your feelings."
    else:
        return "Thanks for sharing your feelings."

# Predict sentiment and generate a response
user_message = "I feel so anxious about my exams."
predicted_sentiment = predict_sentiment(user_message)
response = generate_response(predicted_sentiment)

print(f"User message: {user_message}")
print(f"Predicted sentiment: {predicted_sentiment}")
print(f"Response: {response}")


NameError: name 'model' is not defined